In [1]:
from src.audioProcessing import splitOnSilence

In [2]:
from os import listdir
from os.path import isfile, join

In [3]:
import re

In [15]:
f'{mypath}/{file}'

'./dataset/recordings/stage-1/converted/Corvus-corax_178458.mp3'

In [14]:
chunks = splitOnSilence(f'{mypath}/{file}')

In [4]:
mypath = './dataset/recordings/stage-1/converted'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for i,file in enumerate(files):
    fileID = int(re.findall('[0-9]+.',file)[0][:-1])
    chunks = splitOnSilence(f'{mypath}/{file}')
    print(f'Splitting {file}: file {i+1} out of {len(files)}')
    for chunk in chunks:
        # Define windows with overlap
        # ...
        # For each window:      
        sample = window.get_array_of_samples()

Splitting Corvus-corax_168257.mp3: file 1 out of 464
Splitting Corvus-corax_171515.mp3: file 2 out of 464
Splitting Corvus-corax_178458.mp3: file 3 out of 464
